# **Solar Farm Insight**

Exploratory Data Analysis (`EDA`) for MoonLight Energy Solutions to identify optimal locations for solar farm installations, focusing on sustainability and operational efficiency through data-driven insights.

### **Install Dependencies**

In [1]:
!pip3 install -r ../requirements/local.txt -qqq

### **Import Libraries**

In [2]:
# Set the path to the parent directory
import sys
import pathlib

import pandas as pd

sys.path.append("..")

from scripts.setup_data import extract_data

## **Dataset Overview**
The dataset provides time-series data to analyze solar energy potential and environmental conditions, aiding in site selection for solar farm installations.

In [3]:
# Dataset path
RAW_DATASET_PATH = pathlib.Path("../data/data.zip")
EXTRACTION_OUTPUT_PATH = RAW_DATASET_PATH.parent.parent

DATASET_PATH = RAW_DATASET_PATH.parent

### **Extract dataset**

In [4]:
# Extract the data
extract_data(RAW_DATASET_PATH, EXTRACTION_OUTPUT_PATH)

# Verify intallation
print(*list((EXTRACTION_OUTPUT_PATH / "data").glob("*.csv")), sep="\n")

../data/sierraleone-bumbuna.csv
../data/togo-dapaong_qc.csv
../data/benin-malanville.csv


### **Load the dataset**

In [5]:
# Load Benin data
bn_df = pd.read_csv(
    DATASET_PATH / "benin-malanville.csv", parse_dates=["Timestamp"], index_col="Timestamp"
)

# Load Sierraleone data
sl_df = pd.read_csv(
    DATASET_PATH / "sierraleone-bumbuna.csv", parse_dates=["Timestamp"], index_col="Timestamp"
)

# Load Togo data
tg_df = pd.read_csv(
    DATASET_PATH / "togo-dapaong_qc.csv", parse_dates=["Timestamp"], index_col="Timestamp"
)

In [6]:
print("Benin:", bn_df.shape)
bn_df.head()

Benin: (525600, 18)


,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
Timestamp,,,,,,,,,,,,,,,,,,
2021-08-09 00:01:00,-1.2,-0.2,-1.1,0.0,0.0,26.2,93.4,0.0,0.4,0.1,122.1,0.0,998,0,0.0,26.3,26.2,NaN
2021-08-09 00:02:00,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.6,0.0,0.0,0.0,0.0,0.0,998,0,0.0,26.3,26.2,NaN
2021-08-09 00:03:00,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.7,0.3,1.1,0.5,124.6,1.5,997,0,0.0,26.4,26.2,NaN
2021-08-09 00:04:00,-1.1,-0.1,-1.0,0.0,0.0,26.2,93.3,0.2,0.7,0.4,120.3,1.3,997,0,0.0,26.4,26.3,NaN
2021-08-09 00:05:00,-1.0,-0.1,-1.0,0.0,0.0,26.2,93.3,0.1,0.7,0.3,113.2,1.0,997,0,0.0,26.4,26.3,NaN


In [7]:
print("Sierra Leone:", sl_df.shape)
sl_df.head()

Sierra Leone: (525600, 18)


,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
Timestamp,,,,,,,,,,,,,,,,,,
2021-10-30 00:01:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.1,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
2021-10-30 00:02:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
2021-10-30 00:03:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
2021-10-30 00:04:00,-0.7,0.0,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.1,22.3,22.6,NaN
2021-10-30 00:05:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN


In [8]:
print("Togo:", tg_df.shape)
tg_df.head()

Togo: (525600, 18)


,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
Timestamp,,,,,,,,,,,,,,,,,,
2021-10-25 00:01:00,-1.3,0.0,0.0,0.0,0.0,24.8,94.5,0.9,1.1,0.4,227.6,1.1,977,0,0.0,24.7,24.4,NaN
2021-10-25 00:02:00,-1.3,0.0,0.0,0.0,0.0,24.8,94.4,1.1,1.6,0.4,229.3,0.7,977,0,0.0,24.7,24.4,NaN
2021-10-25 00:03:00,-1.3,0.0,0.0,0.0,0.0,24.8,94.4,1.2,1.4,0.3,228.5,2.9,977,0,0.0,24.7,24.4,NaN
2021-10-25 00:04:00,-1.2,0.0,0.0,0.0,0.0,24.8,94.3,1.2,1.6,0.3,229.1,4.6,977,0,0.0,24.7,24.4,NaN
2021-10-25 00:05:00,-1.2,0.0,0.0,0.0,0.0,24.8,94.0,1.3,1.6,0.4,227.5,1.6,977,0,0.0,24.7,24.4,NaN
